In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 700)

In [2]:
# import master dataset
master_df = pd.read_pickle('./../data/master_dataset.pkl')
print(master_df.shape)
display(master_df.head())

(1132970, 28)


,new_id,old_id,img_path,img,correct_img_date,date_placed_ARF,PMI_days,year,age_at_death,race,sex,autopsy,est_weight_lb,est_weight_kg,est_stature_in,est_stature_cm,cadaver_stature_cm,cadaver_weight_lbs,date_placed_ARF_freezer,true_BP,pred_BP,pred_BP_conf,true_SOD_G,BP_of_true_SOD_G,pred_SOD_G,pred_SOD_G_conf,true_SOD_M,BP_of_true_SOD_M
0,000,UT01-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.01.JPG,00000121.01.JPG,2018-01-21,2018-01-21,0.0,2018,73.0,white,Male,False,160.0,72.562358,70.0,177.8,169.0,150.0,2018-01-04,NaN,stake,100.00,NaN,NaN,NaN,NaN,NaN,NaN
1,000,UT01-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.02.JPG,00000121.02.JPG,2018-01-21,2018-01-21,0.0,2018,73.0,white,Male,False,160.0,72.562358,70.0,177.8,169.0,150.0,2018-01-04,NaN,fullbody,85.51,NaN,NaN,NaN,NaN,NaN,NaN
2,000,UT01-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.03.JPG,00000121.03.JPG,2018-01-21,2018-01-21,0.0,2018,73.0,white,Male,False,160.0,72.562358,70.0,177.8,169.0,150.0,2018-01-04,NaN,fullbody,90.52,NaN,NaN,NaN,NaN,NaN,NaN
3,000,UT01-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.04.JPG,00000121.04.JPG,2018-01-21,2018-01-21,0.0,2018,73.0,white,Male,False,160.0,72.562358,70.0,177.8,169.0,150.0,2018-01-04,NaN,fullbody,99.50,NaN,NaN,NaN,NaN,NaN,NaN
4,000,UT01-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.05.JPG,00000121.05.JPG,2018-01-21,2018-01-21,0.0,2018,73.0,white,Male,False,160.0,72.562358,70.0,177.8,169.0,150.0,2018-01-04,NaN,fullbody,43.47,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
master_df.columns

Index(['new_id', 'old_id', 'img_path', 'img', 'correct_img_date',
       'date_placed_ARF', 'PMI_days', 'year', 'age_at_death', 'race', 'sex',
       'autopsy', 'est_weight_lb', 'est_weight_kg', 'est_stature_in',
       'est_stature_cm', 'cadaver_stature_cm', 'cadaver_weight_lbs',
       'date_placed_ARF_freezer', 'true_BP', 'pred_BP', 'pred_BP_conf',
       'true_SOD_G', 'BP_of_true_SOD_G', 'pred_SOD_G', 'pred_SOD_G_conf',
       'true_SOD_M', 'BP_of_true_SOD_M'],
      dtype='object')

In [46]:
master_df[master_df.autopsy == True].shape

(131567, 28)

In [47]:
# drop columns not needed
master_df.drop(['autopsy','est_weight_kg', 'est_stature_cm', 'pred_BP', 'pred_BP_conf', 'pred_SOD_G', 'pred_SOD_G_conf', 
                'true_SOD_M', 'BP_of_true_SOD_M', 'date_placed_ARF_freezer'], axis=1, inplace=True)

In [48]:
# keep only images with known bodypart
master_df2 = master_df[master_df.true_BP.notnull()].copy()
print(master_df2.shape)
display(master_df2.info())

(45930, 18)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 45930 entries, 7327 to 1129576
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   new_id              45930 non-null  object 
 1   old_id              45930 non-null  object 
 2   img_path            45930 non-null  object 
 3   img                 45930 non-null  object 
 4   correct_img_date    45930 non-null  object 
 5   date_placed_ARF     45930 non-null  object 
 6   PMI_days            45930 non-null  float64
 7   year                45930 non-null  int64  
 8   age_at_death        45930 non-null  float64
 9   race                45930 non-null  object 
 10  sex                 45930 non-null  object 
 11  est_weight_lb       36864 non-null  float64
 12  est_stature_in      41493 non-null  float64
 13  cadaver_stature_cm  40483 non-null  float64
 14  cadaver_weight_lbs  41719 non-null  float64
 15  true_BP             45930 non-null  

None

In [49]:
master_df2.true_BP.unique()

array(['fullbody', 'backside', 'arm', 'foot', 'hips', 'head', 'legs',
       'hand', 'back', 'other', 'plastic', 'torso', 'stake', 'knee',
       'shade'], dtype=object)

In [50]:
# keep only head, torso and limbs images
bp_ls = ['head', 'torso', 'backside', 'arm', 'legs']
master_df3 = master_df2[master_df2["true_BP"].isin(bp_ls)].copy()
print(master_df3.shape)
print(master_df3.true_BP.unique())
display(master_df3.info())

(25323, 18)
['backside' 'arm' 'head' 'legs' 'torso']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 25323 entries, 7337 to 1129576
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   new_id              25323 non-null  object 
 1   old_id              25323 non-null  object 
 2   img_path            25323 non-null  object 
 3   img                 25323 non-null  object 
 4   correct_img_date    25323 non-null  object 
 5   date_placed_ARF     25323 non-null  object 
 6   PMI_days            25323 non-null  float64
 7   year                25323 non-null  int64  
 8   age_at_death        25323 non-null  float64
 9   race                25323 non-null  object 
 10  sex                 25323 non-null  object 
 11  est_weight_lb       20587 non-null  float64
 12  est_stature_in      23311 non-null  float64
 13  cadaver_stature_cm  22277 non-null  float64
 14  cadaver_weight_lbs  23167 non-null  float64


None

In [51]:
# drops rows with missing weight and height values
master_df4 = master_df3[master_df3.est_weight_lb.notnull() | master_df3.cadaver_weight_lbs.notnull() 
                       | master_df3.est_stature_in.notnull() | master_df3.cadaver_stature_cm.notnull()].copy()
master_df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25323 entries, 7337 to 1129576
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   new_id              25323 non-null  object 
 1   old_id              25323 non-null  object 
 2   img_path            25323 non-null  object 
 3   img                 25323 non-null  object 
 4   correct_img_date    25323 non-null  object 
 5   date_placed_ARF     25323 non-null  object 
 6   PMI_days            25323 non-null  float64
 7   year                25323 non-null  int64  
 8   age_at_death        25323 non-null  float64
 9   race                25323 non-null  object 
 10  sex                 25323 non-null  object 
 11  est_weight_lb       20587 non-null  float64
 12  est_stature_in      23311 non-null  float64
 13  cadaver_stature_cm  22277 non-null  float64
 14  cadaver_weight_lbs  23167 non-null  float64
 15  true_BP             25323 non-null  object 
 16 

In [52]:
master_df4.sample(10)

,new_id,old_id,img_path,img,correct_img_date,date_placed_ARF,PMI_days,year,age_at_death,race,sex,est_weight_lb,est_stature_in,cadaver_stature_cm,cadaver_weight_lbs,true_BP,true_SOD_G,BP_of_true_SOD_G
470447,598,UT68-14D,/da1_data/icputrd/arf/mean.js/public/anau_img3/598/59800920.22.JPG,59800920.22.JPG,2014-09-20,2014-07-23,59.0,2014,69.0,white,Male,190.0,70.0,NaN,194.0,head,NaN,NaN
743034,a48,UT01-13D,/da1_data/icputrd/arf/mean.js/public/anau_img3/a48/a4800413.11.JPG,a4800413.11.JPG,2013-04-13,2013-01-23,80.0,2013,23.0,white,Female,NaN,63.0,157.0,140.0,legs,NaN,NaN
1003636,e08,UT110-11D,/da1_data/icputrd/arf/mean.js/public/anau_img3/e08/e0810303.10.JPG,e0810303.10.JPG,2012-03-03,2011-12-26,68.0,2011,71.0,white,Male,NaN,NaN,185.0,NaN,backside,NaN,NaN
670051,952,UT80-12D,/da1_data/icputrd/arf/mean.js/public/anau_img3/952/95210212.09.JPG,95210212.09.JPG,2013-02-12,2012-11-14,90.0,2012,60.0,white,Male,160.0,67.0,160.5,103.0,legs,NaN,NaN
1004875,e08,UT110-11D,/da1_data/icputrd/arf/mean.js/public/anau_img3/e08/e0810410.31.JPG,e0810410.31.JPG,2012-04-10,2011-12-26,106.0,2011,71.0,white,Male,NaN,NaN,185.0,NaN,backside,NaN,NaN
349534,282,UT80-13D,/da1_data/icputrd/arf/mean.js/public/anau_img3/282/28210407.30.JPG,28210407.30.JPG,2014-04-07,2013-12-19,109.0,2013,57.0,white,Female,160.0,65.0,162.0,149.0,head,NaN,NaN
1004565,e08,UT110-11D,/da1_data/icputrd/arf/mean.js/public/anau_img3/e08/e0810401.10.JPG,e0810401.10.JPG,2012-04-01,2011-12-26,97.0,2011,71.0,white,Male,NaN,NaN,185.0,NaN,legs,NaN,NaN
1003018,e08,UT110-11D,/da1_data/icputrd/arf/mean.js/public/anau_img3/e08/e0810209.08.JPG,e0810209.08.JPG,2012-02-09,2011-12-26,45.0,2011,71.0,white,Male,NaN,NaN,185.0,NaN,backside,NaN,NaN
740462,a48,UT01-13D,/da1_data/icputrd/arf/mean.js/public/anau_img3/a48/a4800127.09.JPG,a4800127.09.JPG,2013-01-27,2013-01-23,4.0,2013,23.0,white,Female,NaN,63.0,157.0,140.0,legs,NaN,NaN
10658,00b,UT16-16D,/da1_data/icputrd/arf/mean.js/public/anau_img3/00b/00b00712.07.JPG,00b00712.07.JPG,2016-07-12,2016-03-07,127.0,2016,62.0,white,Male,516.0,73.0,183.0,553.0,head,NaN,NaN


In [53]:
# extract donor and date from img column
master_df4['donor_date'] = master_df4.img.str.split('.', expand=True)[0]
display(master_df4.head(3))

,new_id,old_id,img_path,img,correct_img_date,date_placed_ARF,PMI_days,year,age_at_death,race,sex,est_weight_lb,est_stature_in,cadaver_stature_cm,cadaver_weight_lbs,true_BP,true_SOD_G,BP_of_true_SOD_G,donor_date
7337,00b,UT16-16D,/da1_data/icputrd/arf/mean.js/public/anau_img3/00b/00b00308.18.JPG,00b00308.18.JPG,2016-03-08,2016-03-07,1.0,2016,62.0,white,Male,516.0,73.0,183.0,553.0,backside,NaN,NaN,00b00308
7357,00b,UT16-16D,/da1_data/icputrd/arf/mean.js/public/anau_img3/00b/00b00308.38.JPG,00b00308.38.JPG,2016-03-08,2016-03-07,1.0,2016,62.0,white,Male,516.0,73.0,183.0,553.0,arm,NaN,NaN,00b00308
7363,00b,UT16-16D,/da1_data/icputrd/arf/mean.js/public/anau_img3/00b/00b00308.44.JPG,00b00308.44.JPG,2016-03-08,2016-03-07,1.0,2016,62.0,white,Male,516.0,73.0,183.0,553.0,backside,NaN,NaN,00b00308


In [55]:
# split into individual bodypart dataframes
head_df = master_df4[master_df4.true_BP == 'head'].copy()
torso_df = master_df4[(master_df4.true_BP == 'torso') | (master_df4.true_BP == 'backside')].copy()
limbs_df = master_df4[(master_df4.true_BP == 'arm') | (master_df4.true_BP == 'legs')].copy()
print(head_df.shape, torso_df.shape, limbs_df.shape)

(7187, 19) (5371, 19) (12765, 19)


In [56]:
# align dataframes on donor_date column
head_torso_merge = pd.merge(head_df , torso_df[['donor_date', 'img','true_SOD_G', 'BP_of_true_SOD_G']], 
                            how='inner', on='donor_date')
print(head_torso_merge.shape)
display

head_torso_limbs_merge = pd.merge(head_torso_merge, limbs_df[['donor_date', 'img','true_SOD_G', 'BP_of_true_SOD_G']], 
                                  how='inner', on='donor_date')
print(head_torso_limbs_merge.shape)
head_torso_limbs_merge.head()

(14988, 22)
(147526, 25)


,new_id,old_id,img_path,img_x,correct_img_date,date_placed_ARF,PMI_days,year,age_at_death,race,sex,est_weight_lb,est_stature_in,cadaver_stature_cm,cadaver_weight_lbs,true_BP,true_SOD_G_x,BP_of_true_SOD_G_x,donor_date,img_y,true_SOD_G_y,BP_of_true_SOD_G_y,img,true_SOD_G,BP_of_true_SOD_G
0,00b,UT16-16D,/da1_data/icputrd/arf/mean.js/public/anau_img3/00b/00b00329.23.JPG,00b00329.23.JPG,2016-03-29,2016-03-07,22.0,2016,62.0,white,Male,516.0,73.0,183.0,553.0,head,NaN,NaN,00b00329,00b00329.04.JPG,NaN,NaN,00b00329.08.JPG,NaN,NaN
1,00b,UT16-16D,/da1_data/icputrd/arf/mean.js/public/anau_img3/00b/00b00329.23.JPG,00b00329.23.JPG,2016-03-29,2016-03-07,22.0,2016,62.0,white,Male,516.0,73.0,183.0,553.0,head,NaN,NaN,00b00329,00b00329.14.JPG,NaN,NaN,00b00329.08.JPG,NaN,NaN
2,00b,UT16-16D,/da1_data/icputrd/arf/mean.js/public/anau_img3/00b/00b00329.23.JPG,00b00329.23.JPG,2016-03-29,2016-03-07,22.0,2016,62.0,white,Male,516.0,73.0,183.0,553.0,head,NaN,NaN,00b00329,00b00329.15.JPG,NaN,NaN,00b00329.08.JPG,NaN,NaN
3,00b,UT16-16D,/da1_data/icputrd/arf/mean.js/public/anau_img3/00b/00b00329.23.JPG,00b00329.23.JPG,2016-03-29,2016-03-07,22.0,2016,62.0,white,Male,516.0,73.0,183.0,553.0,head,NaN,NaN,00b00329,00b00329.27.JPG,NaN,NaN,00b00329.08.JPG,NaN,NaN
4,00b,UT16-16D,/da1_data/icputrd/arf/mean.js/public/anau_img3/00b/00b00329.23.JPG,00b00329.23.JPG,2016-03-29,2016-03-07,22.0,2016,62.0,white,Male,516.0,73.0,183.0,553.0,head,NaN,NaN,00b00329,00b00329.28.JPG,NaN,NaN,00b00329.08.JPG,NaN,NaN


In [57]:
# keep only one donor_date
final_df = head_torso_limbs_merge.drop_duplicates(subset=['donor_date']).copy()
display(final_df.info())
display(final_df.describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 872 entries, 0 to 147523
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   new_id              872 non-null    object 
 1   old_id              872 non-null    object 
 2   img_path            872 non-null    object 
 3   img_x               872 non-null    object 
 4   correct_img_date    872 non-null    object 
 5   date_placed_ARF     872 non-null    object 
 6   PMI_days            872 non-null    float64
 7   year                872 non-null    int64  
 8   age_at_death        872 non-null    float64
 9   race                872 non-null    object 
 10  sex                 872 non-null    object 
 11  est_weight_lb       609 non-null    float64
 12  est_stature_in      732 non-null    float64
 13  cadaver_stature_cm  770 non-null    float64
 14  cadaver_weight_lbs  728 non-null    float64
 15  true_BP             872 non-null    object 
 16  true_

None

,PMI_days,year,age_at_death,est_weight_lb,est_stature_in,cadaver_stature_cm,cadaver_weight_lbs,true_SOD_G_x,true_SOD_G_y,true_SOD_G
count,872.000000,872.000000,872.000000,609.000000,732.000000,770.000000,728.000000,14.000000,13.000000,7.000000
mean,49.435780,2013.175459,66.018349,194.686371,67.395765,168.298896,179.155220,3.357143,3.230769,3.142857
std,41.534521,1.251477,22.197312,50.948295,3.980732,11.547129,58.098781,1.336306,0.832050,0.899735
min,0.000000,2011.000000,21.000000,95.000000,60.000000,148.000000,60.000000,2.000000,2.000000,2.000000
25%,18.000000,2013.000000,55.000000,170.000000,64.000000,157.000000,140.000000,3.000000,3.000000,3.000000
50%,37.000000,2013.000000,71.000000,200.000000,66.000000,165.750000,181.000000,3.000000,3.000000,3.000000
75%,73.000000,2014.000000,84.000000,200.000000,71.000000,178.000000,222.000000,3.000000,4.000000,3.000000
max,243.000000,2016.000000,95.000000,516.000000,75.000000,193.000000,553.000000,6.000000,5.000000,5.000000


In [58]:
# rename and drop columns 
head_torso_limbs_merge.rename(columns={"img_x": "img_head", "true_SOD_G_x": "true_SOD_G_head",
                                      "img_y": "img_torso", "true_SOD_G_y": "true_SOD_G_torso",
                                      "img": "img_limbs", "true_SOD_G": "true_SOD_G_limbs"}, inplace=True)
head_torso_limbs_merge.drop(['BP_of_true_SOD_G_x', 'BP_of_true_SOD_G_y', 'BP_of_true_SOD_G'], axis=1, inplace=True)
display(head_torso_limbs_merge.head())
display(head_torso_limbs_merge.info())

,new_id,old_id,img_path,img_head,correct_img_date,date_placed_ARF,PMI_days,year,age_at_death,race,sex,est_weight_lb,est_stature_in,cadaver_stature_cm,cadaver_weight_lbs,true_BP,true_SOD_G_head,donor_date,img_torso,true_SOD_G_torso,img_limbs,true_SOD_G_limbs
0,00b,UT16-16D,/da1_data/icputrd/arf/mean.js/public/anau_img3/00b/00b00329.23.JPG,00b00329.23.JPG,2016-03-29,2016-03-07,22.0,2016,62.0,white,Male,516.0,73.0,183.0,553.0,head,NaN,00b00329,00b00329.04.JPG,NaN,00b00329.08.JPG,NaN
1,00b,UT16-16D,/da1_data/icputrd/arf/mean.js/public/anau_img3/00b/00b00329.23.JPG,00b00329.23.JPG,2016-03-29,2016-03-07,22.0,2016,62.0,white,Male,516.0,73.0,183.0,553.0,head,NaN,00b00329,00b00329.14.JPG,NaN,00b00329.08.JPG,NaN
2,00b,UT16-16D,/da1_data/icputrd/arf/mean.js/public/anau_img3/00b/00b00329.23.JPG,00b00329.23.JPG,2016-03-29,2016-03-07,22.0,2016,62.0,white,Male,516.0,73.0,183.0,553.0,head,NaN,00b00329,00b00329.15.JPG,NaN,00b00329.08.JPG,NaN
3,00b,UT16-16D,/da1_data/icputrd/arf/mean.js/public/anau_img3/00b/00b00329.23.JPG,00b00329.23.JPG,2016-03-29,2016-03-07,22.0,2016,62.0,white,Male,516.0,73.0,183.0,553.0,head,NaN,00b00329,00b00329.27.JPG,NaN,00b00329.08.JPG,NaN
4,00b,UT16-16D,/da1_data/icputrd/arf/mean.js/public/anau_img3/00b/00b00329.23.JPG,00b00329.23.JPG,2016-03-29,2016-03-07,22.0,2016,62.0,white,Male,516.0,73.0,183.0,553.0,head,NaN,00b00329,00b00329.28.JPG,NaN,00b00329.08.JPG,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 147526 entries, 0 to 147525
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   new_id              147526 non-null  object 
 1   old_id              147526 non-null  object 
 2   img_path            147526 non-null  object 
 3   img_head            147526 non-null  object 
 4   correct_img_date    147526 non-null  object 
 5   date_placed_ARF     147526 non-null  object 
 6   PMI_days            147526 non-null  float64
 7   year                147526 non-null  int64  
 8   age_at_death        147526 non-null  float64
 9   race                147526 non-null  object 
 10  sex                 147526 non-null  object 
 11  est_weight_lb       106502 non-null  float64
 12  est_stature_in      137884 non-null  float64
 13  cadaver_stature_cm  125922 non-null  float64
 14  cadaver_weight_lbs  134945 non-null  float64
 15  true_BP             147526 non-nul

None

In [ ]:
# convert time related features to datetime
subset_df['correct_img_date'] = pd.to_datetime(subset_df['correct_img_date'], format='%Y-%m-%d')
subset_df['date_placed_ARF'] = pd.to_datetime(subset_df['date_placed_ARF'], format='%Y-%m-%d')
display(subset_df.info())
display(subset_df.head())

In [ ]:
# write to pickle file 
head_torso_limbs_merge.to_pickle('filtered_master_dataset.pkl')